## Faster R-CNN



In [1]:
"""
It is an object detection model that identifies objects in an image and draws bounding boxes around them, while also classifying what those objects are.


Here we do two things simultaneously, that is Classification and Regression. Classification to identify class of Object and Regression to find the
cordinates of bounding boxes around it.

Components:

Backbone Network : Any pretrained Model - Feature Extractor


https://medium.com/@fractal.ai/guide-to-build-faster-rcnn-in-pytorch-42d47cb0ecd3
"""

'\nIt is an object detection model that identifies objects in an image and draws bounding boxes around them, while also classifying what those objects are.\n\n\nHere we do two things simultaneously, that is Classification and Regression. Classification to identify class of Object and Regression to find the\ncordinates of bounding boxes around it.\n\nComponents:\n\nBackbone Network : Any pretrained Model - Feature Extractor\n\n\nhttps://medium.com/@fractal.ai/guide-to-build-faster-rcnn-in-pytorch-42d47cb0ecd3\n'

In [2]:
import torch
import torchvision

In [3]:
dummy_img = torch.zeros((1, 3, 800, 800)).float() # batch size, channels, height, width

In [4]:
bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # [y1, x1, y2, x2] format

In [5]:
labels = torch.LongTensor([6,8])

In [6]:
sub_sample = 16

In [7]:
# List all layers of VGG16, which we will be using as Backbone for our RPN

In [8]:
model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 65.5MB/s]


In [9]:
"""
The VGG16 model is around 500mb in sizes coz it has total of 138 million parameters, and each parameter
is of float32 (4 bytes) size, and 138 million x 4 byte is around 500MBs.
"""

'\nThe VGG16 model is around 500mb in sizes coz it has total of 138 million parameters, and each parameter\nis of float32 (4 bytes) size, and 138 million x 4 byte is around 500MBs.\n'

In [10]:
"""
Basically model contain two types of layers: Feature Layers and Clasifier Layers
Here we have to do work only with Feature Layers, which are responsible for Extracting Features from the input image.
"""

'\nBasically model contain two types of layers: Feature Layers and Clasifier Layers\nHere we have to do work only with Feature Layers, which are responsible for Extracting Features from the input image.\n'

In [11]:
model.features # Contain all CNN layers, excluding all Fully Connected Layers(Classifier Layers)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [12]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [13]:
"""
Now we have to configure VGG16 model, that matches our required Input and Output sizes. This is why we
have created dummy_img, just to experiment on it.
"""

'\nNow we have to configure VGG16 model, that matches our required Input and Output sizes. This is why we\nhave created dummy_img, just to experiment on it.\n'

In [14]:
fe = list(model.features)

In [15]:
fe

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1

In [16]:
req_features=[]
k = dummy_img.clone()
out_channels = 0
for i in fe:
  k = i(k)
  if k.size()[2] < 800//16:
    break
  req_features.append(i)
  out_channels = k.size()[1]

In [17]:
800//16  # // is integral division or floor diviion

50

In [18]:
len(req_features)

30

In [19]:
out_channels

512

In [20]:
req_features

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1

In [21]:
faster_rcnn_fe_extractor = torch.nn.Sequential(*req_features)
# here * is unpacking operator, this will unpack list for form seprate arguments for nn.Sequential

In [22]:
out_map = faster_rcnn_fe_extractor(dummy_img)

In [23]:
out_map.size()

torch.Size([1, 512, 50, 50])

## Anchor Boxes

In [24]:
"""
So our next task is to define the Anchor Boxes, over feature maps.
Basically, There are anchor boxes, of diffrent shape and sizes, that we are gonna slide over feature maps.
"""

'\nSo our next task is to define the Anchor Boxes, over feature maps.\nBasically, There are anchor boxes, of diffrent shape and sizes, that we are gonna slide over feature maps.\n'

In [25]:
import numpy as np

In [26]:
ratios = [0.5, 1, 2] # diffrent ratio of anchor baxoes, i.e, define the diffrent shapes : tall, square and wide

In [27]:
anchor_scales = [8, 16, 32] # diffrent size of anchor boxes i.e, small, medium and large

In [28]:
anchor_base = np.zeros((len(ratios)*len(anchor_scales), 4), dtype=np.float32)

In [29]:
anchor_base # format : x1, y1, x2, y2

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [30]:
"""
There will be total of 9(3x3) anchor boxes, which will combine all predefined shape and sizes.

Key understandings:

-> One pixel in the 50×50 feature map corresponds to a region of 16×16 pixels in the original image,
as we downsampled our image by the factor of 16.

-> Each pixel in the 50×50 feature map "looks at" a receptive field of 16×16 pixels in the input image.
  So:

    Pixel (0, 0) in the feature map represents image area (0:16, 0:16)

    Pixel (1, 0) represents (16:32, 0:16)

    Pixel (0, 1) represents (0:16, 16:32)

    And so on.

We use this mapping to place anchor boxes in image coordinates.


-> The anchor box center is aligned with the center of the 16×16 region that a feature map pixel corresponds to in the input image.

-> For feature map location (0, 0), the center in image space is at (8, 8)
For (1, 0), center = (8 + 16, 8) = (24, 8)
For (x, y), center = (16 * x + 8, 16 * y + 8)




"""

'\nThere will be total of 9(3x3) anchor boxes, which will combine all predefined shape and sizes.\n\nKey understandings:\n\n-> One pixel in the 50×50 feature map corresponds to a region of 16×16 pixels in the original image,\nas we downsampled our image by the factor of 16.\n\n-> Each pixel in the 50×50 feature map "looks at" a receptive field of 16×16 pixels in the input image.\n  So:\n\n    Pixel (0, 0) in the feature map represents image area (0:16, 0:16)\n\n    Pixel (1, 0) represents (16:32, 0:16)\n\n    Pixel (0, 1) represents (0:16, 16:32)\n\n    And so on.\n\nWe use this mapping to place anchor boxes in image coordinates.\n\n\n-> The anchor box center is aligned with the center of the 16×16 region that a feature map pixel corresponds to in the input image.\n\n-> For feature map location (0, 0), the center in image space is at (8, 8)\nFor (1, 0), center = (8 + 16, 8) = (24, 8)\nFor (x, y), center = (16 * x + 8, 16 * y + 8)\n\n\n\n\n'

In [31]:
sub_sample

16

In [32]:
ctr_y = sub_sample/2
ctr_x = sub_sample/2

In [33]:
(ctr_y, ctr_x)

(8.0, 8.0)

In [34]:
for i in range(len(ratios)):
  for j in range(len(anchor_scales)):
    h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
    w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
    index = i * len(anchor_scales) + j
    anchor_base[index, 0] = ctr_y - h / 2.
    anchor_base[index, 1] = ctr_x - w / 2.
    anchor_base[index, 2] = ctr_y + h / 2.
    anchor_base[index, 3] = ctr_x + w / 2.

In [35]:
anchor_base

array([[ -37.254833,  -82.50967 ,   53.254833,   98.50967 ],
       [ -82.50967 , -173.01933 ,   98.50967 ,  189.01933 ],
       [-173.01933 , -354.03867 ,  189.01933 ,  370.03867 ],
       [ -56.      ,  -56.      ,   72.      ,   72.      ],
       [-120.      , -120.      ,  136.      ,  136.      ],
       [-248.      , -248.      ,  264.      ,  264.      ],
       [ -82.50967 ,  -37.254833,   98.50967 ,   53.254833],
       [-173.01933 ,  -82.50967 ,  189.01933 ,   98.50967 ],
       [-354.03867 , -173.01933 ,  370.03867 ,  189.01933 ]],
      dtype=float32)

In [36]:
"""
Now each pixel represent an image space of 16x16 in which anchor boxes are to be drawn.
We have total of 9 such boxes in each pixel and total of 50x50 pixels in features maps.
So total number of anchor boxes are 9*50*50 which is 22500.


RECALL : The anchor is genrated at each pixel of our Feature Map, and each pixel of Feature Map will be treated
as center for the anchor. Each pixel of 50x50 feature map, represent the 16x16 area of Original Image.

So,

ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)

using this we will find the center point of that 16x16 image space in ouriginal 800x800 image. This is done by:

  ctr_x[i] - 8

"""

'\nNow each pixel represent an image space of 16x16 in which anchor boxes are to be drawn.\nWe have total of 9 such boxes in each pixel and total of 50x50 pixels in features maps.\nSo total number of anchor boxes are 9*50*50 which is 22500.\n\n\nRECALL : The anchor is genrated at each pixel of our Feature Map, and each pixel of Feature Map will be treated\nas center for the anchor. Each pixel of 50x50 feature map, represent the 16x16 area of Original Image.\n\nSo,\n\nctr_x = np.arange(16, (fe_size+1) * 16, 16)\nctr_y = np.arange(16, (fe_size+1) * 16, 16)\n\nusing this we will find the center point of that 16x16 image space in ouriginal 800x800 image. This is done by:\n\n  ctr_x[i] - 8\n\n'

In [37]:
fe_size = (800//16)

In [38]:
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)

In [39]:
ctr_x

array([ 16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192, 208,
       224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400, 416,
       432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608, 624,
       640, 656, 672, 688, 704, 720, 736, 752, 768, 784, 800])

In [40]:
ctr = np.zeros((len(ctr_x) * len(ctr_y), 2), dtype=np.float32)

In [41]:
index = 0
for x in range(len(ctr_x)):
  for y in range(len(ctr_y)):
    ctr[index, 1] = ctr_x[x] - 8
    ctr[index, 0] = ctr_y[y] - 8
    index += 1

In [42]:
len(ctr_x)

50

In [43]:
ctr

array([[  8.,   8.],
       [ 24.,   8.],
       [ 40.,   8.],
       ...,
       [760., 792.],
       [776., 792.],
       [792., 792.]], dtype=float32)

In [44]:
ctr.shape # so in total we have 2500 anchor centers

(2500, 2)

In [45]:
"""
Now at each center, we need to generate the anchor boxes.
"""

'\nNow at each center, we need to generate the anchor boxes.\n'

In [46]:
anchors = np.zeros((fe_size*fe_size*9, 4))

In [47]:
anchors

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [48]:
anchors.shape # total 22500 anchor boxes

(22500, 4)

In [49]:
index = 0
for c in ctr:
  ctr_y, ctr_x = c
  for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
      h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
      w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
      anchors[index, 0] = ctr_y - h / 2.
      anchors[index, 1] = ctr_x - w / 2.
      anchors[index, 2] = ctr_y + h / 2.
      anchors[index, 3] = ctr_x + w / 2.
      index += 1

In [50]:
anchors

array([[ -37.254834  ,  -82.50966799,   53.254834  ,   98.50966799],
       [ -82.50966799, -173.01933598,   98.50966799,  189.01933598],
       [-173.01933598, -354.03867197,  189.01933598,  370.03867197],
       ...,
       [ 701.49033201,  746.745166  ,  882.50966799,  837.254834  ],
       [ 610.98066402,  701.49033201,  973.01933598,  882.50966799],
       [ 429.96132803,  610.98066402, 1154.03867197,  973.01933598]])

In [51]:
"""
Let say below given some data, belonging to Ground Truth. Now our next task is to find anchor boxes, that fit
them the most. Faster RCNN has specific guidelines to assiging the labels to the anchor boxes.

There will be two kind of lables: positive and negative.
ox and its corresponding anchor box
the highest iou for each anch
We will be assiging positive lable to two kind of anchor:
  a) The anchor with highest Intersection-over-Union(IoU) with ground truth box.
  b) An anchor that has an IoU overlap higher than 0.7 with ground-truth box.


NOTE : Single Ground truth object may assign positive lables to multiple anchors.

We assign a negative label to a non-positive anchor if its IoU ratio is lower than 0.3 for all ground-truth boxes.


Anchors that are neither positive nor negitive do not contribute to the training objective, i.e anchor baoxes
with IoU in between 0.3 and 0.7, will be ignored. Ignoring them reduces noise in training and improves stability.

"""

bbox = np.asarray([[20,30,400,500],[300,400,500,600]], dtype=np.float32)
labels = np.asarray([6,8], dtype=np.int8) # 0 is for background, not used here

"""
The two labels here represent the two diffent objects, maybe 6 can prepresent a car, and 8 can represent a dog.
0 will be reserved for Background.
"""

'\nThe two labels here represent the two diffent objects, maybe 6 can prepresent a car, and 8 can represent a dog.\n0 will be reserved for Background.\n'

In [52]:
"""
Now we will create a Label array, and initilized it with -1 values. Then calculate IoU at each index
of anchor boxes.
"""

'\nNow we will create a Label array, and initilized it with -1 values. Then calculate IoU at each index\nof anchor boxes.\n'

In [53]:
index_inside = np.where(
    (anchors[:, 0] >= 0) &   # y1 ≥ 0 (top boundary inside image)
    (anchors[:, 1] >= 0) &   # x1 ≥ 0 (left boundary inside image)
    (anchors[:, 2] <= 800) & # y2 ≤ 800 (bottom boundary inside image)
    (anchors[:, 3] <= 800)   # x2 ≤ 800 (right boundary inside image)
)[0]

# now this gives index of all valid anchor boxes, that are inside image boundaries.

In [54]:
anchors[:, 0] # all rows, column 0

array([ -37.254834  ,  -82.50966799, -173.01933598, ...,  701.49033201,
        610.98066402,  429.96132803])

In [55]:
index_inside # is a list of valid anchor box indexes

array([ 1410,  1419,  1428, ..., 21075, 21084, 21093])

In [56]:
index_inside.shape

(8940,)

In [57]:
label = np.empty((len(index_inside),), dtype=np.int32)

In [58]:
label.fill(-1)

In [59]:
label # initilizing label array with all negative labels

array([-1, -1, -1, ..., -1, -1, -1], dtype=int32)

In [60]:
valid_anchor_boxes = anchors[index_inside]

In [61]:
valid_anchor_boxes

array([[ 13.49033201,  10.745166  , 194.50966799, 101.254834  ],
       [ 29.49033201,  10.745166  , 210.50966799, 101.254834  ],
       [ 45.49033201,  10.745166  , 226.50966799, 101.254834  ],
       ...,
       [573.49033201, 698.745166  , 754.50966799, 789.254834  ],
       [589.49033201, 698.745166  , 770.50966799, 789.254834  ],
       [605.49033201, 698.745166  , 786.50966799, 789.254834  ]])

In [62]:
valid_anchor_boxes.shape

(8940, 4)

In [63]:
"""
Now for each anchor boxes, we need to calculate IoU with each ground truth.

Since we have 2 ground truth objects and total of 8940 valid anchor boxes, we should get an
array of shape (8940, 2) as output
"""

'\nNow for each anchor boxes, we need to calculate IoU with each ground truth.\n\nSince we have 2 ground truth objects and total of 8940 valid anchor boxes, we should get an\narray of shape (8940, 2) as output\n'

In [64]:
ious = np.empty((len(valid_anchor_boxes),2), dtype=np.float32)
ious.fill(0)

In [65]:
ious

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [66]:
for num1, i in enumerate(valid_anchor_boxes):
    ya1, xa1, ya2, xa2 = i
    anchor_area = (ya2 - ya1) * (xa2 - xa1) # length*width
    for num2, j in enumerate(bbox):
        yb1, xb1, yb2, xb2 = j
        box_area = (yb2- yb1) * (xb2 - xb1)

        inter_x1 = max([xb1, xa1])
        inter_y1 = max([yb1, ya1])
        inter_x2 = min([xb2, xa2])
        inter_y2 = min([yb2, ya2])

        if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
            iter_area = (inter_y2 - inter_y1) * (inter_x2 - inter_x1)
            iou = iter_area / (anchor_area+ box_area - iter_area)
        else:
            iou = 0.
        ious[num1, num2] = iou

In [67]:
ious

array([[0.06811669, 0.        ],
       [0.07083762, 0.        ],
       [0.07083762, 0.        ],
       ...,
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]], dtype=float32)

In [68]:
# threshold = 0.7
# matches = np.argwhere(ious > threshold)

# # Display results
# for i, j in matches:
#     print(f"Anchor {i} and GT Box {j} -> IoU: {ious[i, j]:.4f}")

In [69]:
"""
Now we will find two things here:
    the highest iou for each gt_box and its corresponding anchor box
    the highest iou for each anchor box and its corresponding ground truth box
"""

'\nNow we will find two things here:\n    the highest iou for each gt_box and its corresponding anchor box\n    the highest iou for each anchor box and its corresponding ground truth box\n'

In [70]:
gt_argmax_ious = ious.argmax(axis=0)

In [71]:
gt_argmax_ious # index of highest element

array([2262, 5620])

In [72]:
gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]

In [73]:
gt_max_ious # Max Ground Truth IoUs

array([0.68130493, 0.61035156], dtype=float32)

In [74]:
argmax_ious = ious.argmax(axis=1)
print(argmax_ious.shape)
print(argmax_ious)
max_ious = ious[np.arange(len(index_inside)), argmax_ious]
print(max_ious)

(8940,)
[0 0 0 ... 0 0 0]
[0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]


In [75]:
gt_argmax_ious = np.where(ious == gt_max_ious)[0]

In [76]:
gt_argmax_ious

array([2262, 2508, 5620, 5628, 5636, 5644, 5866, 5874, 5882, 5890, 6112,
       6120, 6128, 6136, 6358, 6366, 6374, 6382])

In [77]:
pos_iou_threshold  = 0.7
neg_iou_threshold = 0.3

In [78]:
label[max_ious < neg_iou_threshold] = 0 # assiging negative labels(0)

In [79]:
label[gt_argmax_ious] = 1 # assiging positive lablels
label[max_ious >= pos_iou_threshold] = 1

## Training RPN

In [81]:
"""
Each mini-batch arises from a single image that contains many positive and negitive example anchors,
but this will bias towards negitive samples as they are dominate. Instead, we randomly sample 256 anchors
in an image to compute the loss function of a mini-batch, where the sampled positive and negative anchors
have a ratio of up to 1:1. If there are fewer than 128 positive samples in an image, we pad the mini-batch
with negitive ones.
"""

'\nEach mini-batch arises from a single image that contains many positive and negitive example anchors, \nbut this will bias towards negitive samples as they are dominate. Instead, we randomly sample 256 anchors \nin an image to compute the loss function of a mini-batch, where the sampled positive and negative anchors \nhave a ratio of up to 1:1. If there are fewer than 128 positive samples in an image, we pad the mini-batch \nwith negitive ones.\n'

In [82]:
pos_ratio = 0.5
n_sample = 256

In [83]:
n_pos = pos_ratio * n_sample

In [84]:
n_pos

128.0

In [1]:
# Will Continue this later on